# Pip this row

In [ ]:
pip install pyvi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.9 MB/s eta 0:00:00


# Import Data & Process Data

In [ ]:
import pandas as pd
import numpy as np
import re
from pyvi import ViTokenizer

In [ ]:
df = pd.read_csv('./data/PHQ9_Student_Depression_Dataset.csv')

In [ ]:
def preprocess_text(text: str) -> str:
    text = re.sub(r'\b(\w+)(?:\s+\1\b)+', r'\1', text)
    text = re.sub(r'\s+', ' ', text.strip())
    text = text.replace('_', ' ')
    text = ViTokenizer.tokenize(text)
    return text

In [ ]:
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
print(len(df))
print("-------------------------------------------")
print(df['label'].value_counts())

250
-------------------------------------------
label
4    69
3    53
2    45
0    45
1    38
Name: count, dtype: int64


# Data Augmentation

In [ ]:
pip install googletrans

In [ ]:
pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 22.8 MB/s eta 0:00:00


In [ ]:
from googletrans import Translator
import random
import nlpaug.augmenter.word as naw
from typing import List, Tuple
import asyncio

In [ ]:
#Init translator for Back Translation function
translator = Translator()

In [ ]:
#Init dict for Synonym Replacement func
SYNONYM_DICT = {
    "mệt_mỏi": ["uể_oải", "rã_rời", "kiệt_sức", "đuối_sức", "bơ_phờ", "lừ_đừ", "mỏi_mệt", "thiếu_sinh_lực", "yếu_ớt", "oải"],
    "vô_vọng": ["tuyệt_vọng", "hết_hy_vọng", "mất_hy_vọng", "chán_nản", "bi_quan", "tuyệt_đường", "bế_tắc", "không_lối_thoát", "đen_tối", "thất_vọng"],
    "trống_rỗng": ["hụt_hẫng", "trống_trải", "vô_vị", "nhạt_nhẽo", "thiếu_sức_sống", "rỗng_tuếch", "không_có_gì", "khô_khan", "tẻ_nhạt", "lặng_lẽ"],
    "tự_ti": ["thiếu_tự_tin", "mắc_cỡ", "e_dè", "rụt_rè", "khép_nép", "không_dám", "yếu_đuối", "nhút_nhát", "tự_ti_mặc_cảm", "tệ_hơn_người"],
    "buồn_bã": ["u_sầu", "sầu_não", "thảm_não", "ưu_phiền", "muộn_phiền", "chán_chường", "đau_khổ", "khổ_sở", "rầu_rĩ", "ảm_đạm"],
    "khó_khăn": ["trắc_trở", "gian_nan", "vất_vả", "khổ_sở", "chật_vật", "gập_ghềnh", "hiểm_trở", "gay_go", "nan_giải", "thách_thức"],
}

In [ ]:
# Synonym Replacement function
def synonym_replacement(text: str, n: int = 0.2) -> str:
    words = text.split()
    new_words = words.copy()
    n_replace = max(1, int(len(words) * n))
    replace_indices = random.sample(range(len(words)), n_replace)

    for idx in replace_indices:
        word = new_words[idx]
        if word in SYNONYM_DICT and random.random() > 0.5:
            new_words[idx] = random.choice(SYNONYM_DICT[word])

    return ' '.join(new_words)

In [ ]:
# Back Translation function
async def back_translation(text: str) -> str:
    try:
        translated = await translator.translate(text, src='vi', dest='en')
        back_translated = await translator.translate(translated.text, src='en', dest='vi')
        return back_translated.text
    except Exception as e:
        print(f"Error in back translation: {e}")
        return text

In [ ]:
async def run_back_translation(text: str) -> str:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        return await back_translation(text)
    else:
        return await loop.run_until_complete(back_translation(text))

In [ ]:
# Sample mixing function
def sample_mixing(text1: str, text2: str) -> str:
    sentences1 = text1.split('.')
    sentences2 = text2.split('.')
    sentences1 = [s.strip() for s in sentences1 if s.strip()]
    sentences2 = [s.strip() for s in sentences2 if s.strip()]

    if not sentences1 or not sentences2:
        return text1

    mixed_sentences = random.sample(sentences1, len(sentences1)//2) + random.sample(sentences2, len(sentences2)//2)
    return '. '.join(mixed_sentences)

In [ ]:
# Augmented data function
async def augment_data(df: pd.DataFrame, target_size: int = 500) -> pd.DataFrame:
    augmented_data = []
    current_size = len(df)

    samples_per_class = target_size // df['label'].nunique()

    for label in df['label'].unique():
        class_df = df[df['label'] == label]
        current_class_size = len(class_df)
        samples_needed = samples_per_class - current_class_size

        if samples_needed <= 0:
            augmented_data.append(class_df)
            continue

        for _ in range(samples_needed):
            sample = class_df.sample(1).iloc[0]
            text = sample['text']
            method = random.choice(['synonym', 'back_translation', 'sample_mixing'])

            if method == 'synonym':
                new_text = synonym_replacement(text)
            elif method == 'back_translation':
                new_text = await run_back_translation(text)
            else:
                other_sample = class_df.sample(1).iloc[0]['text']
                new_text = sample_mixing(text, other_sample)

            new_sample = sample.copy()
            new_sample['text'] = new_text
            new_sample['ID'] = f"aug_{len(augmented_data)}_{label}"
            augmented_data.append(pd.DataFrame([new_sample]))

        augmented_data.append(class_df)

    augmented_df = pd.concat(augmented_data, ignore_index=True)
    augmented_df = augmented_df.sample(frac=1, random_state=42).reset_index(drop=True)
    return augmented_df

In [ ]:
augmented_df = await augment_data(df, target_size=500)

In [ ]:
print(len(augmented_df))
print("-------------------------------------------")
print(augmented_df['label'].value_counts())
print("-------------------------------------------")
print(augmented_df.info())

500
-------------------------------------------
label
2    100
4    100
1    100
0    100
3    100
Name: count, dtype: int64
-------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              500 non-null    object
 1   text            500 non-null    object
 2   PHQ-9 Score     500 non-null    int64 
 3   Severity Level  500 non-null    object
 4   label           500 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 19.7+ KB
None


In [ ]:
augmented_df.to_csv('./data/PHQ9_Student_Depression_Dataset_augmented.csv', index=False)

# Build Classification Model- PhoBERT

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
# from underthesea import word_tokenize
from tqdm import tqdm

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    augmented_df["text"], augmented_df["label"], test_size=0.2, random_state=42
)

label_encoder = LabelEncoder()
label_encoder.fit(augmented_df["label"])

LabelEncoder()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", num_labels=len(label_encoder.classes_)
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_texts.tolist())
test_encodings = tokenize_function(test_texts.tolist())

In [ ]:
class DepressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DepressionDataset(train_encodings, train_labels.tolist())
test_dataset = DepressionDataset(test_encodings, test_labels.tolist())

In [ ]:
label_mapping = {
    0: "Minimal",
    1: "Mild",
    2: "Moderate",
    3: "Moderately Severe",
    4: "Severe"
}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=50 ,
    weight_decay=0.01,
    logging_dir="./logs",
    eval_strategy="epoch",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)

accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Accuracy: {accuracy}")
f1 = f1_score(test_labels, predicted_labels, average='weighted')
print(f"F1-score: {f1}")

Accuracy: 0.85
F1-score: 0.8537690058479532


In [ ]:
model.save_pretrained("./phobert_depression_model")
tokenizer.save_pretrained("./phobert_depression_model")

('./phobert_depression_model/tokenizer_config.json',
 './phobert_depression_model/special_tokens_map.json',
 './phobert_depression_model/vocab.txt',
 './phobert_depression_model/bpe.codes',
 './phobert_depression_model/added_tokens.json')

In [ ]:
import os

In [ ]:
save_directory = "./model"

os.makedirs(save_directory, exist_ok=True)

# Save
model_path = os.path.join(save_directory, "model_state_dict.pt")
torch.save(model.state_dict(), model_path)

tokenizer.save_pretrained(save_directory)

print(f"Model state dictionary saved to {model_path}")
print(f"Tokenizer saved to {save_directory}")

print("Files in save directory:", os.listdir(save_directory))

Model state dictionary saved to /content/drive/MyDrive/Project_ChatBot/model/model_state_dict.pt
Tokenizer saved to /content/drive/MyDrive/Project_ChatBot/model
Files in save directory: ['vocab.txt', '.ipynb_checkpoints', 'model_state_dict.pt', 'tokenizer_config.json', 'special_tokens_map.json', 'added_tokens.json', 'bpe.codes']


In [ ]:
model.load_state_dict(torch.load("./model/model_state_dict.pt"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained("./model/")

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


In [ ]:
label_mapping = {0: "Minimal", 1: "Mild", 2: "Moderate", 3: "Moderately Severe", 4: "Severe"}
text = "tôi không thể ngủ vào ban đêm và luôn cảm thấy mệt mỏi"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
inputs = {key: val.to(device) for key, val in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs["logits"]
    predicted_label = torch.argmax(logits, dim=1).item()
print(f"Predicted: {label_mapping[predicted_label]}")

Predicted: Moderately Severe


# Compare to original model and Gemini

In [ ]:
import google.generativeai as genai

In [ ]:
def evaluate_model(model, dataset, device):
    model.eval()
    predictions = []
    true_labels = []
    for batch in DataLoader(dataset, batch_size=16):
        inputs = {key: val.to(device) for key, val in batch.items() if key != "labels"}
        labels = batch["labels"].to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs["logits"]
            preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='macro')
    return predictions, accuracy, f1

In [ ]:
base_model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", num_labels=len(label_encoder.classes_)
)
base_model.to(device)
base_preds, base_accuracy, base_f1 = evaluate_model(base_model, test_dataset, device)
print("\nBase Model (No Fine-tuning):")
print(f"Accuracy: {base_accuracy:.4f}")
print(f"F1 Scor: {base_f1:.4f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Base Model (No Fine-tuning):
Accuracy: 0.2800
F1 Scor: 0.2411


In [ ]:
def gemini_predict(text):
    prompt = f"""
    Phân loại mức độ trầm cảm dựa trên văn bản sau (tiếng Việt):
    Văn bản: "{text}"
    Các mức độ: Minimal, Mild, Moderate, Moderately Severe, Severe
    Trả về chỉ tên mức độ (ví dụ: Severe)
    """
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        response = model.generate_content(prompt)
        label = response.text.strip()
        reverse_label_mapping = {v.lower(): k for k, v in label_mapping.items()}
        return reverse_label_mapping.get(label.lower(), 0)
    except Exception as e:
        print(f"Error with Gemini API: {e}")
        return 0

In [ ]:
gemini_preds = []
for text in test_texts:
    pred = gemini_predict(text)
    gemini_preds.append(pred)

Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


...

Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


Error with Gemini API: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.


In [ ]:
gemini_accuracy = accuracy_score(test_labels, gemini_preds)
gemini_f1 = f1_score(test_labels, gemini_preds, average='macro')
print("\nGemini API:")
print(f"Accuracy: {gemini_accuracy:.4f}")
print(f"F1 Score: {gemini_f1:.4f}")


Gemini API:
Accuracy: 0.2300
F1 Score: 0.1207
